<a href="https://colab.research.google.com/github/joony0512/Deep_Learning_Class/blob/main/Part6/P6_Ch02_CH02_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation - RNN Approach


In [1]:
import os
import cv2
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from glob import glob
from pprint import pprint

import tensorflow_hub as hub
import tensorflow as tf
from tensorflow import keras

In [2]:
model = keras.models.load_model(
    '/content/drive/MyDrive/딥러닝 정주행/P6_Ch02.동영상처리실습/ucf11_weights/10-0.78.hdf5',
    custom_objects={'KerasLayer' : hub.KerasLayer}
)



In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              5919312   
                                                                 
 dense (Dense)               (None, 11)                14091     
                                                                 
Total params: 5933403 (22.63 MB)
Trainable params: 5872795 (22.40 MB)
Non-trainable params: 60608 (236.75 KB)
_________________________________________________________________


In [4]:
# feature extracter 만 사용할것이기 때문에 dense를 거둬야한다

feature_extractor = keras.Sequential(
    [
        keras.Input(shape = (256,256,3)),
        model.layers[0]
    ], name = 'feature_extractor'
)

feature_extractor.summary()

Model: "feature_extractor"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              5919312   
                                                                 
Total params: 5919312 (22.58 MB)
Trainable params: 5858704 (22.35 MB)
Non-trainable params: 60608 (236.75 KB)
_________________________________________________________________


In [ ]:
# os.mkdir('/content/drive/MyDrive/딥러닝 정주행/P6_Ch02.동영상처리실습/UCF11_updated_numpy')
# os.mkdir('/content/drive/MyDrive/딥러닝 정주행/P6_Ch02.동영상처리실습/UCF11_updated_numpy/train')
# os.mkdir('/content/drive/MyDrive/딥러닝 정주행/P6_Ch02.동영상처리실습/UCF11_updated_numpy/valid')

In [13]:
max_frames = 10
SAVE_DIR = '/content/drive/MyDrive/딥러닝 정주행/P6_Ch02.동영상처리실습/UCF11_updated_numpy/'
train_df = pd.read_csv('/content/drive/MyDrive/딥러닝 정주행/P6_Ch02.동영상처리실습/ucf11_train_vid.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/딥러닝 정주행/P6_Ch02.동영상처리실습/ucf11_valid_vid.csv')

In [ ]:
for i, elem in tqdm(train_df.iterrows(), total=len(train_df)):
    label = elem['label']
    cap = cv2.VideoCapture(elem['file_path'])
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_ = frame.copy()
        frame_ = cv2.cvtColor(frame_, cv2.COLOR_BGR2RGB)
        frame_ = cv2.resize(frame_, (256, 256))
        frame_ = frame_.astype('float32')
        frame_ = frame_ / 255.

        frames.append(frame_)

        if len(frames) == max_frames:
            break

    cap.release()

    frames = np.array(frames)
    features = feature_extractor.predict(frames)

    file_name = SAVE_DIR + f'train/{label}_{i}.npy'
    np.save(file_name, features)


In [14]:
print(len(glob(SAVE_DIR + 'train/*')))

220


In [ ]:
for i, elem in tqdm(valid_df.iterrows(), total=len(valid_df)):
    label = elem['label']
    cap = cv2.VideoCapture(elem['file_path'])
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_ = frame.copy()
        frame_ = cv2.cvtColor(frame_, cv2.COLOR_BGR2RGB)
        frame_ = cv2.resize(frame_, (256, 256))
        frame_ = frame_.astype('float32')
        frame_ = frame_ / 255.

        frames.append(frame_)

        if len(frames) == max_frames:
            break

    cap.release()

    frames = np.array(frames)
    features = feature_extractor.predict(frames)

    file_name = SAVE_DIR + f'valid/{label}_{i}.npy'
    np.save(file_name, features)


In [16]:
print(len(glob(SAVE_DIR + 'valid/*')))

55
